In [1]:
import os 
import re
from random import randint, random
from textwrap import dedent

In [2]:
import numpy as np
from math import ceil

In [171]:
DOC_PATH = os.path.join('..','rvv-intrinsic-doc-master','intrinsic_funcs')
VLEN = 128

def random_line():
    md_files = os.listdir(DOC_PATH)
    filename = [f for f in md_files if f.startswith("07")][0]
    filename = os.path.join(DOC_PATH,filename)
    filesize = os.stat(filename).st_size
    with open(filename,'r') as f:
        ret = ""
        while not re.findall(';\s+$',ret):
            file_pointer = randint(0,filesize)
            f.seek(file_pointer)
            f.readline()
            ret =  f.readline()
        return ret

In [258]:
class RvvType():
    SIZE_DICT = {'mf8':1/8,'mf4':1/4,'mf2':1/2,'m1':1,'m2':2,'m4':4,'m8':8,}
    attr = ['basetype','sew','lmul','vl','bytesize','abbr']
    def __init__(self, string):
        if not string.startswith('v'):
            raise RuntimeError("%s not a rvv type" %string)        
        self.string = string
        self._parse()
        assert all([getattr(self,attr,None) for attr in self.attr]), "not all attr defined"
        
    def __eq__(self,other):
        return self.string == other.string
    def __hash__(self):
        return hash(self.string)
    
    def __repr__(self):
        dtype = 'rvv vector type %s:\n' %self.string
        attrs = '\n'.join([name+':'+ str(getattr(self,name,None)) for name in self.attr])
        return dtype+attrs
        
    def _abbr_rule(self,match):
        dtype = match.group(0)
        if dtype=='uint':
            return 'u'
        elif dtype=='int':
            return 'i'
        elif dtype=='float':
            return 'f'     
        
    def _parse(self):
        match = re.search('(?P<basetype>(u?int\d+)|(float\d+))(?P<lmul>mf?\d)',self.string)
        self.lmul = lmul = match.group('lmul')
        regsize = self.SIZE_DICT.get(lmul)*VLEN
        regsize = int(regsize)
        self.bytesize = regsize//8
        basetype = match.group('basetype')
        self.abbr = re.sub('(?P<match>(u?int)|(float))',self._abbr_rule,basetype)+lmul
        self.basetype = basetype+'_t'
        self.sew = basesize = re.search('(?P<size>\d+)',basetype).group('size')
        self.vl = int(regsize)//int(basesize)
    
    @property
    def declare(self):
        return dedent('''\
        {basetype} *{{op}}_base = ({basetype}*){{addr}};
        '''.format(basetype=self.basetype))   
    
    @property
    def context(self):
        return dedent('''\
        {} {{op}} = vle{}_v_{}({{op}}_base,{});
        '''.format(self.string, self.sew, self.abbr,self.vl))
    
    @property
    def post_context(self):
        return dedent('''\
        vse{}_v_{}({{op}}_base,{{op}},{});
        '''.format(self.sew, self.abbr,self.vl))

In [259]:
class RvvBool(RvvType):
    LMUL_DICT = {v:k for k,v in RvvType.SIZE_DICT.items()}
    attr = ['basetype','n','vl','bytesize','abbr']
    def __init__(self,string):
        RvvType.__init__(self,string)
        
    def _parse(self):
        self.n = n = re.search('(?P<n>\d+)',self.string).group('n')
        self.abbr = 'b%s' %n
        self.vl = vlmax = VLEN//int(n)
        self.bytesize = vlmax//8 + int((vlmax%8)!=0)
        self.basetype = 'uint8_t'
        self.sew = 1
        self.lmul = 1
    
    @property
    def context(self):
        lmul = self.vl*8/VLEN
        lmul = self.LMUL_DICT[lmul]
        return dedent('''\
        {vbool_t} {{op}} = vle1_v_b{n}({{op}}_base,{vl});   
        uint8{lmul}_t vec_{{op}} = vmv_v_x_u8{lmul}(0,{vl});
        vec_{{op}} = vmerge_vxm({{op}},vec_{{op}},1,{vl});
        _Bool {{op}}_bool[{vl}];
        vse8_v_u8{lmul}({{op}}_bool,vec_{{op}},{vl});
        '''.format(lmul=lmul,vl=self.vl,vbool_t = self.string,n=self.n))
    
    @property
    def post_context(self):
        lmul = self.vl*8/VLEN
        lmul = self.LMUL_DICT[lmul]
        return dedent('''\
        vuint8{lmul}_t vec_{{op}} = vmv_v_x_u8{lmul}(0,{vl});
        vec_{{op}} = vmerge_vxm_u8{lmul}({{op}},vec_{{op}},1,{vl});
        vse8_v_u8{lmul}({{op}}_base,vec_{{op}},{vl});
        '''.format(lmul=lmul,vl=self.vl,vbool_t = self.string,n=self.n))
    

In [260]:
class ScalarType():
    TYPEDEF = {'size_t':'uint32_t',}
    def __init__(self,string):
        if string.startswith('v'):
            raise RuntimeError("%s is not a scalar type" %string)
        self.string = string
        self.basetype = string
        self.alias = self.TYPEDEF.get(string,string)
        self.bytesize = self._sizeof(self.alias)
        self.attr = ['bytesize','alias']
        
    def __repr__(self):
        dtype = 'scalar type %s:\n' %self.string
        attrs = '\n'.join([name+':'+ str(getattr(self,name,None)) for name in self.attr])
        return dtype+attrs
    
    def _sizeof(self,typename):
        return np.dtype(typename.rstrip('_t')).itemsize
    
    @property
    def declare(self):
        return dedent('''\
        {dtype} *{{op}}_base = ({dtype}*){{addr}};
        '''.format(dtype=self.string))
    @property
    def context(self):
        return dedent('''\
        {dtype} {{op}} = *{{op}}_base;
        '''.format(dtype=self.string))

In [261]:
def parse_dtype(dtype:str):
    if dtype.startswith('vbool'):
        return RvvBool(dtype)
    elif dtype.startswith('v'):
        return RvvType(dtype)
    else:
        return ScalarType(dtype)
    
def parse_declaration(line):
    ret, func, rest_of_line = line.split(" ", 2)
    match = re.search('(?P<ops>(?<=\().+(?=\)))',rest_of_line)
    op_pairs = match.group('ops').split(',')
    op_pairs = [op.split() for op in op_pairs]
    operands = {arg:dtype for dtype,arg in op_pairs}
    #return {'ret':ret,'func':func,'ops':operands}
    return [ret,func,operands]

In [262]:
line = random_line()
line

'vfloat16mf2_t vfcvt_f_x_v_f16mf2 (vint16mf2_t src, size_t vl);\n'

In [263]:
line = 'vbool16_t vmadc_vx_u16m1_b16 (vuint16m1_t op1, uint16_t op2, size_t vl);'

In [264]:
ret_name,func_name,ops_name = parse_declaration(line)

In [306]:
class AddrDispensor():
    def __init__(self,addr_begin:int):
        self.begin = addr_begin
        self.addr = addr_begin
        
    def get_addr(self,len_btye):
        assert (len_btye>0), 'addr increment cannot <=0'
        addr = self.addr
        self.addr += len_btye
        self.addr = ceil(self.addr/8)*8
        return addr
    def get_begin(self):
        return self.begin
    def get_lenb(self):
        return self.addr - self.begin

In [307]:
ad = AddrDispensor(0x2000)

In [308]:
ret = parse_dtype(ret_name)
str_declare = ''
str_context = ''
str_post_context = ''
str_declare += dedent('''\
    {dtype} *golden_base = ({dtype}*){addr_g};
    {dtype} *actual_base = ({dtype}*){addr_a};
    '''.format(dtype=ret.basetype,
               addr_g=ad.get_addr(ret.bytesize),
               addr_a=ad.get_addr(ret.bytesize)))
str_post_context += ret.post_context.format(op='actual')

In [309]:
for name,dtype in ops_name.items():
    dtype = parse_dtype(dtype)
    str_declare += dtype.declare.format(op=name,addr=ad.get_addr(dtype.bytesize))
    str_context += dtype.context.format(op=name)

In [310]:
v_op = ' '.join([ret_name , 'actual=' , func_name ,'(' , ', '.join(ops_name) , ');'])

In [313]:
RANMEM_TEMPLATE=dedent('''\
    void gen_random_mem(size_t mem_start, size_t n){
        uint8_t *mem = (uint8_t*)mem_start;
        for(int i=0;i<n;i++){
            mem[i] = rand();
        }
    }
    ''')
C_TEMPLATE=dedent('''
    #include "string.h"
    #include "riscv_vector.h"
    #include "random.h"
    int main(){{
    /*** DECLARE ***/
    {0}
    gen_random_mem({ad_begin},{lenb});
    /*** CONTEXT ***/
    {1}
    /***vector operation***/
    {2}
    /***POST CONTEXT***/
    {3}
    return 0;
    }}
    ''')
print(C_TEMPLATE.format(str_declare,str_context,v_op,str_post_context,ad_begin=ad.get_begin(),lenb=ad.get_lenb()))


#include "string.h"
#include "riscv_vector.h"
#include "random.h"
int main(){
/*** DECLARE ***/
uint8_t *golden_base = (uint8_t*)8192;
uint8_t *actual_base = (uint8_t*)8200;
uint16_t *op1_base = (uint16_t*)8208;
uint16_t *op2_base = (uint16_t*)8224;
size_t *vl_base = (size_t*)8232;

gen_random_mem(8192,48);
/*** CONTEXT ***/
vuint16m1_t op1 = vle16_v_u16m1(op1_base,8);
uint16_t op2 = *op2_base;
size_t vl = *vl_base;

/***vector operation***/
vbool16_t actual= vmadc_vx_u16m1_b16 ( op1, op2, vl );
/***POST CONTEXT***/
vuint8mf2_t vec_actual = vmv_v_x_u8mf2(0,8);
vec_actual = vmerge_vxm_u8mf2(actual,vec_actual,1,8);
vse8_v_u8mf2(actual_base,vec_actual,8);

return 0;
}



In [314]:
func_name

'vmadc_vx_u16m1_b16'

In [220]:
parse_dtype(ops_name['op1'])

rvv vector type vuint16m1_t:
basetype:uint16_t
sew:16
lmul:m1
vl:8
bytesize:16
abbr:u16m1

In [221]:
TEMPLATE_SCALAR_VMADC=dedent('''\
    size_t bitwidth = sizeof(op2)*8;
    for(int i=0;i<{vl};i++){{
        {basetype} op1 = op1_base[i];
        {basetype} sum = op1_base[i] + op2; 
        golden_base[i] = (sum^(op1&op2))>>(bitwidth-1);
    }}''')

In [169]:
print(TEMPLATE_SCALAR_VMADC)

size_t bitwidth = sizeof(op2)*8;
for(int i=0;i<{vl};i++){{
    {basetype} op1 = op1_base[i];
    {basetype} sum = op1_base[i] + op2; 
    golden_base[i] = (sum^(op1&op2))>>(bitwidth-1);
}}


In [271]:
print(TEMPLATE_SCALAR_VMADC.format(basetype=eval("ops_name['op2']"),vl=10))

size_t bitwidth = sizeof(op2)*8;
for(int i=0;i<10;i++){
    uint16_t op1 = op1_base[i];
    uint16_t sum = op1_base[i] + op2; 
    golden_base[i] = (sum^(op1&op2))>>(bitwidth-1);
}
